In [1]:
using HDF5
using PyPlot
using Statistics

In [2]:
U = h5open("v1_layer3/U_avg.h5", "r") do file
    read(file, "matrix")end

V = h5open("v1_layer3/V_avg.h5", "r") do file
    read(file, "matrix")end

M = h5open("v1_layer3/M_avg.h5", "r") do file
    read(file, "matrix")end

xgrid = h5open("v1_layer3/xgrid_matrix", "r") do file
        read(file, "matrix")end

ygrid = h5open("v1_layer3/ygrid_matrix", "r") do file
        read(file, "matrix")end

63×63×394 Array{Float32, 3}:
[:, :, 1] =
   16.0    16.0    16.0    16.0    16.0  …    16.0    16.0    16.0    16.0
   32.0    32.0    32.0    32.0    32.0       32.0    32.0    32.0    32.0
   48.0    48.0    48.0    48.0    48.0       48.0    48.0    48.0    48.0
   64.0    64.0    64.0    64.0    64.0       64.0    64.0    64.0    64.0
   80.0    80.0    80.0    80.0    80.0       80.0    80.0    80.0    80.0
   96.0    96.0    96.0    96.0    96.0  …    96.0    96.0    96.0    96.0
  112.0   112.0   112.0   112.0   112.0      112.0   112.0   112.0   112.0
  128.0   128.0   128.0   128.0   128.0      128.0   128.0   128.0   128.0
  144.0   144.0   144.0   144.0   144.0      144.0   144.0   144.0   144.0
  160.0   160.0   160.0   160.0   160.0      160.0   160.0   160.0   160.0
  176.0   176.0   176.0   176.0   176.0  …   176.0   176.0   176.0   176.0
  192.0   192.0   192.0   192.0   192.0      192.0   192.0   192.0   192.0
  208.0   208.0   208.0   208.0   208.0      208.0   208.0 

In [3]:
replace!(U, NaN => 0)
replace!(V, NaN => 0)

63×63×394 Array{Float32, 3}:
[:, :, 1] =
 0.0       0.0        0.0       0.0       …   0.0        0.0          0.0
 0.0       0.0        0.0       0.0           0.0        0.0          0.0
 0.0       0.0        0.0       0.0           0.0        0.0          0.0
 0.0       0.0        0.0       0.0           0.0        0.0          0.0
 0.0       0.0        0.0       0.0           0.0        0.0          0.0
 0.0       0.0        0.0       0.0       …   0.0        0.0          0.0
 0.0       0.0        0.0       0.0           0.0        0.0          0.0
 0.0       0.0        0.0       0.0           0.0        0.0          0.0
 0.0       0.0        0.0       0.0           0.0        0.0          0.0
 0.0       0.0        0.0       0.0           0.0        0.0          0.0
 0.0       0.0        0.0       0.0       …   0.0        0.0          0.0
 0.0       0.0        0.0       0.0           0.0        0.0          0.0
 0.0       0.0        0.0       0.0           0.0        0.0          0

# Original function modified into 2D.
from https://github.com/Marc-3d/multi_quickPIV/blob/main/src/postprocessing/postprocessing.jl

In [10]:
function PIVtrajectories_grid(U::Array{Float32, 3}, V::Array{Float32, 3}, T0::Int64, T1::Int64; 
                                subregion=( 1:-1, 1:-1 ), step=(1,1), scale=(1,1))
    # The staring points are in all points in the vector matrix. 
    # Automatticlly returns all points in vector field from subregion the tracking data.
    # step determine the distance between the starting points. 
    # scale can rescale the length of the arrows. PIV has the tendence to underestimate the movement velocity.

    # Length of the each axis of the vector field. 
    dims  = ( length( 1:size(U,1) ), length( 1:size(U,2) ) ) 

    # The user can limit the simulation to a certain subregion of the vector field. 
    sampling_region = [ length( subregion[i] ) == 0 ? (2:step[i]:dims[i]-1) : Base.StepRange( subregion[i].start, step[i], subregion[i].stop ) for i in 1:2 ];
    scale = (typeof(scale)<:Number) ? (scale,scale) : scale

    numT = T1 - T0;
    TrajectoriesY = zeros( Float32, numT, prod( length.(sampling_region) ) ) # Have to be changed if manuelly giving the starting coordenates.
    TrajectoriesX = zeros( Float32, numT, prod( length.(sampling_region) ) )

    pidx = 0; 
    for x in sampling_region[2], y in sampling_region[1] # Could customize the sampling region through manuelly giving the x and y coordinates. 

        pidx += 1; 

        # Placing a new particle inside the vector-field. This is done by 
        # randomly picking a random position withing the vector-field. 
        starting_pos = (y,x); 

        # Recording the starting position in the first timepoints in the trajectory tables for point $pidx. 
        TrajectoriesY[ 1, pidx ] = Float32( starting_pos[1] )
        TrajectoriesX[ 1, pidx ] = Float32( starting_pos[2] )

        # Sampling the translation at the current ( starting ) position
        dY = Float32( scale[1] * U[ starting_pos..., T0 ] )
        dX = Float32( scale[2] * V[ starting_pos..., T0 ] )
        
        # moving forward in time, from T0 to T1
        for t in 2:numT

            # New_pos = previous position + translation (dU,dV,dW); 
            updated_pos = ( TrajectoriesY[t-1, pidx], TrajectoriesX[t-1, pidx] ) .+ ( dY, dX ); 

            # Recording the updated position in the trajectory tables
            TrajectoriesY[t,pidx] = updated_pos[1]
            TrajectoriesX[t,pidx] = updated_pos[2]

            # Obtaining the integer index of the updated position
            int_updated_pos = round.( Int64, updated_pos ); 

            # If the (integer) updated position is out of the coordinates of the vector field, stop
            if any( int_updated_pos .< 1 ) || any( int_updated_pos .> dims ) 
                TrajectoriesY[ t:end, pidx ] .= TrajectoriesY[ t-1, pidx ]
                TrajectoriesX[ t:end, pidx ] .= TrajectoriesX[ t-1, pidx ]
                break
            end

            # Sampling the translation at the (integer) updated position
            dY = Float32( scale[1] * U[ int_updated_pos..., T0+t-1 ] )
            dX = Float32( scale[2] * V[ int_updated_pos..., T0+t-1 ] )
        end
    end

    return TrajectoriesY, TrajectoriesX
end

PIVtrajectories_grid (generic function with 1 method)

In [13]:
PIVtrajectories_grid(U, V, 100, 150)

(Float32[2.0 3.0 … 61.0 62.0; 2.9314337 3.9197855 … 60.079018 61.0; … ; 23.841679 18.77805 … 57.606934 53.145416; 23.841679 18.77805 … 57.606934 54.00926], Float32[2.0 2.0 … 62.0 62.0; 1.9554796 2.8981066 … 60.027077 59.0356; … ; 10.030341 8.217827 … 48.276554 52.87551; 10.030341 8.217827 … 48.276554 52.762077])

# Modification with given sampling regions

Problems: 

1. seems to have large rounding errors
2. The x and y positions are implemented with vector field matrix positions instead of the image pixel positions! Quite unconvient for application.
3. Use only the vector direction of the directed neighboured IA center. No averaging of neighbour fields.

In [4]:
function PIVtrajectories_grid_given_cor(U::Array{Float32, 3}, V::Array{Float32, 3}, T0::Int64, T1::Int64,
                                        x_cor::Vector{Int64}, y_cor::Vector{Int64}, scale::Tuple{Float64, Float64})
    # The staring points are in all points in the vector matrix. 
    # Automatticlly returns all points in vector field from subregion the tracking data.
    # step determine the distance between the starting points. 
    # scale can rescale the length of the arrows. PIV has the tendence to underestimate the movement velocity.

    if length(x_cor) != length(y_cor)
        print("x and y positions should have the same lengths!")
        
    else

        # Length of the each axis of the vector field. 
        dims  = ( length( 1:size(U,1) ), length( 1:size(U,2) ) ) 
    
        numT = T1 - T0 + 1;
        TrajectoriesY = zeros( Float32, numT,  length(x_cor)) 
        TrajectoriesX = zeros( Float32, numT, length(x_cor) )
    
        for pidx in 1:length(x_cor) # Could customize the sampling region through manuelly giving the x and y coordinates. 

            x = x_cor[pidx]
            y = y_cor[pidx]
    
            # Placing a new particle inside the vector-field. This is done by 
            # randomly picking a random position withing the vector-field. 
            starting_pos = (y,x); 
    
            # Recording the starting position in the first timepoints in the trajectory tables for point $pidx. 
            TrajectoriesY[ 1, pidx ] = Float32( starting_pos[1] )
            TrajectoriesX[ 1, pidx ] = Float32( starting_pos[2] )
    
            # Sampling the translation at the current ( starting ) position
            dY = Float32( scale[1] * U[ starting_pos..., T0 ] )
            dX = Float32( scale[2] * V[ starting_pos..., T0 ] )
            
            # moving forward in time, from T0 to T1
            for t in 2:numT

                # New_pos = previous position + translation (dU,dV,dW); 
                updated_pos = ( TrajectoriesY[t-1, pidx], TrajectoriesX[t-1, pidx] ) .+ ( dY, dX )
    
                # Recording the updated position in the trajectory tables
                TrajectoriesY[t,pidx] = updated_pos[1]
                TrajectoriesX[t,pidx] = updated_pos[2]
    
                # Obtaining the integer index of the updated position
                int_updated_pos = round.(Int64, updated_pos)
    
                # If the (integer) updated position is out of the coordinates of the vector field, stop
                if any( int_updated_pos .< 1 ) || any( int_updated_pos .> dims ) 
                    TrajectoriesY[ t:end, pidx ] .= TrajectoriesY[ t-1, pidx ]
                    TrajectoriesX[ t:end, pidx ] .= TrajectoriesX[ t-1, pidx ]
                    break
                end
    
                # Sampling the translation at the (integer) updated position
                dY = Float32( scale[1] * U[ int_updated_pos..., T0+t-1 ] )
                dX = Float32( scale[2] * V[ int_updated_pos..., T0+t-1 ] )

            end
        end
    end

    return TrajectoriesY, TrajectoriesX
end

PIVtrajectories_grid_given_cor (generic function with 1 method)

In [5]:
y, x = PIVtrajectories_grid_given_cor(U, V, 10, 20, [20], [40], (0.2, 0.2))
y

11×1 Matrix{Float32}:
 40.0
 40.550224
 40.91236
 41.332188
 41.81813
 42.03409
 42.03409
 42.03409
 42.250046
 42.466003
 42.68794

In [6]:
function single_track(U::Array{Float32, 3}, V::Array{Float32, 3}, t_array::Vector{Vector{Int64}}, 
                    x_cor_array::Vector{Vector{Int64}}, y_cor_array::Vector{Vector{Int64}}, scale::Tuple{Float64, Float64})
    
    track_x = Vector{Float32}[]
    track_y = Vector{Float32}[]
    
    num_track = length(x_cor)
    
    for i in 1:num_track
        
        t0 = round(Int64, t_array[i][1])
        t1 = round(Int64, last(t_array[i]))

        x_cor = round.(Int64, x_cor_array[i]./16) # transform the image coordinate into vector field coordinate.
        y_cor = round.(Int64, y_cor_array[i]./16)

        y, x = PIVtrajectories_grid_given_cor(U, V, t0, t1, x_cor, y_cor, scale)

        push!(track_x, vec(x)) # Row vector. If want coloumn vector use vec(x).
        push!(track_y, vec(y)) 
    end
    return track_x, track_y
end

single_track (generic function with 1 method)

In [7]:
track_id = [1, 9, 15, 21, 22, 25, 38, 39, 44]
t_track = [[81, 113], [134, 172], [160, 174], [102, 135], [124, 137], [86, 192], [72, 118], [116, 120], [232, 271]]
x_cor = [[885], [758], [894], [881], [708], [132], [914], [818], [902]]
y_cor = [[412], [462], [166], [653], [439], [114], [365], [238], [534]]
x_track, y_track = single_track(U, V, t_track, x_cor,y_cor, (0.1, 0.1))

(Vector{Float32}[[55.0, 55.031944, 55.06389, 55.086952, 55.110016, 55.13613, 55.162247, 55.209454, 55.266785, 55.339092  …  55.311283, 55.235332, 55.15938, 55.055954, 54.930946, 54.828434, 54.73119, 54.633945, 54.57085, 54.523457], [47.0, 47.122517, 47.20108, 47.21274, 47.2147, 47.212738, 47.2244, 47.23606, 47.247723, 47.279804  …  49.238712, 49.238712, 49.250378, 49.263847, 49.277317, 49.364647, 49.378117, 49.389782, 49.401447, 49.414917], [56.0, 56.18932, 56.32199, 56.450375, 56.584995, 56.773926, 56.962856, 57.151787, 57.340717, 57.481712, 57.622707, 57.699852, 57.769382, 57.838913, 57.85498], [55.0, 54.89463, 54.78926, 54.68389, 54.57852, 54.57852, 54.57852, 54.57852, 54.57852, 54.57852  …  54.58892, 54.58892, 54.58892, 54.58892, 54.58892, 54.58892, 54.58892, 54.58892, 54.58892, 54.58892], [44.0, 44.003155, 44.08476, 44.166363, 44.247967, 44.247967, 44.247967, 44.247967, 44.247967, 44.247967, 44.247967, 44.247967, 44.247967, 44.247967], [8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0,

In [2]:
# y_track, x_track = PIVtrajectories_grid_given_cor(U, V, 1, 300, [10, 20, 30], [10, 20, 30])

Tracking starting pixel positions from experimental data: 
Manuelly noted stop last tracking at t=271

p1: x = 249, y = 460, t=17

p76: x = 185, y = 546, t=22

p32: x = 108, y = 392, t = 31

p8: x = 519, y = 577, t=49

p33: x = 120, y = 437, t=51

p2: x=244, y=498, t=52

p80, x = 45, y=237, t=53

p84: x=11, y=298, t=59

p20: x=378, y=345, t=59

p74: x=294, y=230, t=57

p5: x=37, y=141, t=61

p41/42: x=69, y =507, t=64

p27: x = 182, y=54, t=67

p30: x=501, y=338, t=79

In [87]:
#Early stage + Short time periderm tracking samples.
#x_start_img = [67, 12, 295, 58, 379, 181, 117, 46, 518]
#y_start_img = [481, 297, 231, 210, 346, 54, 432, 237, 586]
#t0=50
#t1=60

# Middle stage + Long time periderm tracking samples.
#x_start_img = [686, 553, 664, 642, 501, 100]
#y_start_img = [274, 384, 308, 502, 338, 85]
#t0 = 78
#t1 = 127

# Later stage short track
# x_start_img = [669, 684, 687, 406, 444]
# y_start_img = [123, 141, 150, 463, 518]
# t0 = 170
# t1 = 180 

# Later stage
# x_start_img = [584, 413, 413, 481, 473, 676]
# y_start_img = [523, 561, 563, 655, 652, 402]
# t0 = 230
# t1 = 260

In [88]:
# x_start = round.(Int64, x_start_img./16)
# y_start = round.(Int64, y_start_img./16)

In [89]:
# y_track, x_track = PIVtrajectories_grid_given_cor(U, V, t0, t1, x_start, y_start)
# t_track = [i for i in t0:t1]

In [8]:
h5open("track", "w") do file
    write(file, "track_id", track_id)
    for i in 1:length(track_id)
        id = track_id[i]
        write(file, "x_track_$id", x_track[i])
        write(file, "y_track_$id", y_track[i])
        write(file, "t_track_$id", t_track[i])
    end
end

In [ ]:
# h5open("track", "w") do file
#     write(file, "track_id", track_id)
#     write(file, "x_track", x_track)
# end

In [12]:
h5open("track", "r") do file
    read(file, "x_track_9")
end

38-element Vector{Float32}:
 47.0
 48.22516
 48.22516
 48.22516
 48.22516
 48.22516
 48.22516
 48.22516
 48.22516
 48.22516
 48.22516
 48.22516
 48.22516
  ⋮
 48.240143
 48.240143
 48.240143
 48.240143
 48.240143
 48.240143
 48.240143
 48.240143
 48.240143
 48.240143
 48.240143
 48.240143